In [1]:
#! /usr/bin/env python3
"""

Creates individual usermod_dirs for each PLUMBER2 site with shell_commands

"""

#  Import libraries
from __future__ import print_function

import os
import sys
import tqdm
import logging
import argparse
import subprocess

import pandas as pd

In [4]:
# Big ugly function to create usermod_dirs for each site
def write_usermods(lat,lon,site,start_year,end_year,
                  start_date,start_tod,atm_ncpl,stop_n):    

    site_dir = os.path.join('../../cime_config/usermods_dirs/PLUMBER2/',site)
    #TODO add overwrite option here

    if not os.path.isdir(site_dir):
        os.makedirs(site_dir, exist_ok=True)    
        
    # create files in each directory
    include = os.path.join(site_dir,'include_user_mods')
    iFile = open(include, 'w') # or 'a' to add text instead of truncate
    iFile.write('../defaults')
    iFile.close()
    
    LAIstream = '/glade/work/oleson/PLUMBER2/input_files/${PLUMBER2SITE}/LAI_stream_${PLUMBER2SITE}_'+ \
                str(start_year)+'-'+str(end_year)+'.nc'
    shell = os.path.join(site_dir,'shell_commands')
    sFile = open(shell, 'w') # or 'a' to add text instead of truncate
    sFile.write(
        './xmlchange PLUMBER2SITE='+site + '\n' \
        './xmlchange PTS_LON='+str(lon) + '\n' \
        './xmlchange PTS_LAT='+str(lat) + '\n' \
        './xmlchange RUN_STARTDATE='+str(start_date) + '\n' \
        './xmlchange DATM_YR_ALIGN='+str(start_year) + '\n' \
        './xmlchange DATM_YR_START='+str(start_year) + '\n' \
        './xmlchange DATM_YR_END='+str(end_year) + '\n' \
        './xmlchange START_TOD='+str(start_tod) + '\n' \
        './xmlchange ATM_NCPL='+str(atm_ncpl) + '\n' \
        '\n' \

        'compset=`./xmlquery COMPSET --value` \n' \
        'CLM_USRDAT_NAME=`./xmlquery CLM_USRDAT_NAME --value` \n' \
        'TEST=`./xmlquery TEST --value` \n' \
        '\n' \

        '# For a transient case run the whole length and do not cycle \n' \
        'if  [[ $compset =~ ^HIST ]]; then \n' \
        '  # Number of years that can be run for the full transient case \n' \
        '  if [[ $TEST != "TRUE" ]]; then  \n' \
        '    ./xmlchange STOP_N='+str(stop_n) + '\n' \
        '  fi \n' \
        'fi \n'  \
        '\n' \

        '# Turn on LAI streams for a SP case \n' \
        'if [[ $compset =~ .*CLM[0-9]+%[^_]*SP.* ]]; then \n' \
        '  echo "stream_fldfilename_lai=\''+LAIstream+'\'" >> user_nl_clm \n' \
        '  echo "model_year_align_lai='+str(start_year) + '" >> user_nl_clm \n' \
        '  echo "stream_year_first_lai='+str(start_year) + '" >> user_nl_clm \n' \
        '  echo "stream_year_last_lai='+str(end_year) + '" >> user_nl_clm \n' \
        'fi \n'      
    )

    sFile.close()
    
    # add baseflow_scalar = 0 to user_nl_clm for wetland sites
    wetland = ["CZ-wet","DE-SfN","FI-Kaa","FI-Lom","RU-Che",\
               "SE-Deg","US-Los","US-Myb","US-Tw4","PL-wet"]
    if any(x == site for x in wetland):
        print('wetland '+site)
        sFile = open(shell, 'a') # or 'a' to add text instead of truncate
        sFile.write(
            '\n' \
            '# set baseflow scalar to zero for wetland site \n' \
            'echo "baseflow_scalar = 0" >> user_nl_clm'
        )
        sFile.close()
    #print(site_dir)

In [5]:
# For now we can just run this as a loop

In [6]:
plumber2_sites = pd.read_csv('PLUMBER2_sites.csv', skiprows=4)  

for i, row in tqdm.tqdm(plumber2_sites.iterrows()):
    lat = row['Lat']
    lon = row['Lon']
    site = row['Site']
    start_year = row['start_year']
    end_year = row['end_year']
    start_date = row['RUN_STARTDATE']
    start_tod = row['START_TOD']
    atm_ncpl = row['ATM_NCPL']
    stop_n = 1+end_year-start_year
    
    write_usermods(lat,lon,site,start_year,end_year,
                  start_date,start_tod,atm_ncpl,stop_n)

57it [00:04, 15.17it/s]

wetland CZ-wet


65it [00:04, 17.19it/s]

wetland DE-SfN


78it [00:05, 29.02it/s]

...

148it [00:09, 12.16it/s]

wetland US-Myb


165it [00:10, 24.01it/s]

wetland US-Tw4


170it [00:10, 16.50it/s]


In [7]:
cat ../../cime_config/usermods_dirs/PLUMBER2/ZM-Mon/shell_commands

./xmlchange PLUMBER2SITE=ZM-Mon
./xmlchange PTS_LON=23.252781
./xmlchange PTS_LAT=-15.437778
./xmlchange START_DATE=2007-12-31
./xmlchange DATM_YR_ALIGN=2008
./xmlchange DATM_YR_START=2008
./xmlchange DATM_YR_END=2008
./xmlchange START_TOD=79200
./xmlchange ATM_NCPL=48

compset=`./xmlquery COMPSET --value` 
CLM_USRDAT_NAME=`./xmlquery CLM_USRDAT_NAME --value` 
TEST=`./xmlquery TEST --value` 

# For a transient case run the whole length and do not cycle 
if  [[ $compset =~ ^HIST ]]; then 
  # Number of years that can be run for the full transient case 
  if [[ $TEST != "TRUE" ]]; then  
    ./xmlchange STOP_N=1
  fi 
fi 

# Turn on LAI streams for a SP case 
if [[ $compset =~ .*CLM[0-9]+%[^_]*SP.* ]]; then 
  echo "stream_fldfilename_lai='/glade/work/oleson/PLUMBER2/input_files/${PLUMBER2SITE}/LAI_stream_${PLUMBER2SITE}_2008-2008.nc'" >> user_nl_clm 
  echo "model_year_align_lai=2008" >> user_nl_clm 
  echo "stream_year_first_lai=2008" >> user_nl_clm 
  echo "stream_year_last_lai=2008" 

In [21]:
cat PLUMBER2_sites.csv

#start_year and end_year will be used to define DATM_YR_ALIGH, DATM_YR_START and DATM_YR_END, and STOP_N in units of nyears.
#RUN_STARTDATE and START_TOD are specified because we are starting at GMT corresponding to local midnight.
#ATM_NCPL is specified so that the time step of the model matches the time interval specified by the atm forcing data.
,Site,Lat,Lon,pft1,pft1-%,pft2,pft2-%,start_year,end_year,RUN_STARTDATE,START_TOD,ATM_NCPL
1,AR-SLu,-33.464802,-66.459808,5,50.00,7,50.00,2010,2010,2010-01-01,10800,48 
2,AT-Neu,47.116669,11.317500,13,100.00,-999,-999.00,2002,2012,2001-12-31,82800,48 
3,AU-ASM,-22.283001,133.248993,1,100.00,-999,-999.00,2011,2017,2010-12-31,54000,48 
4,AU-Cow,-16.238190,145.427155,4,100.00,-999,-999.00,2010,2015,2009-12-31,50400,48 
5,AU-Cpr,-34.002060,140.589127,7,30.00,14,70.00,2011,2017,2010-12-31,46800,48 
6,AU-Ctr,-16.103279,145.446854,4,100.00,-999,-999.00,2010,2017,2009-12-31,50400,48 
7,AU-Cum,-33.613297,150.722473,5,100.00,-999,-999.00,2013,2018,201